# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting info from tensors
* Manipulating Tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try

## Introduction to Tensors


In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.8.2


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimension of our vector
vector.ndim

1

In [6]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7], [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
matrix.ndim

2

In [8]:
# Create a matrix with a different datatype
another_matrix = tf.constant([[10., 7.], [3., 2.], [8., 9.]], dtype=tf.float16)
another_matrix


<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# Check # of dims in another_matrix
another_matrix.ndim

2

In [10]:
# Create a tensor
tensor = tf.constant([[
                       [1, 2, 3,],
                       [4, 5, 6,]
                      ],[
                        [7, 8, 9], 
                        [10, 11, 12],
                      ],[
                        [13, 14, 15], 
                        [16, 17, 18]
                       ]
                      ])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
# Check # of dims in tensor
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [12]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [13]:
# Changing one of the elements in the changeable tensor
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [14]:
# Try changing one of the elements in the unchangeable tensor
# unchangeable_tensor[0].assign(7) # (errors out)
unchangeable_tensor

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>

### Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers.

Random tensors are used when initializing weights for machine learning

In [15]:
# Create 2 random tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility 
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

### Shuffle the order of elements in a tensor

In [16]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[1,2], [3,4], [5,6], [7,8], [9,10]])
tf.random.set_seed(42)
shuffled = tf.random.shuffle(not_shuffled)
not_shuffled, shuffled

(<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
 array([[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10]], dtype=int32)>, <tf.Tensor: shape=(5, 2), dtype=int32, numpy=
 array([[ 5,  6],
        [ 1,  2],
        [ 9, 10],
        [ 7,  8],
        [ 3,  4]], dtype=int32)>)

🔧 **Exercise:** Read through TensorFlow docs on random seed generation: [random/set_seed](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)

Practice writing 5 random tensors and shuffle them.

In [17]:
# Create 2 random tensors
r = tf.random.Generator.from_seed(42) # set seed for reproducibility 
r1 = r.normal(shape=(4, 2))
r2 = r.normal(shape=(5, 2))
r1, r2

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[ 1.4680984 , -0.69443417],
        [-0.56105393, -1.1563786 ],
        [ 0.61561096,  0.67493457],
        [ 1.9547467 ,  0.71566796],
        [ 1.0842259 , -2.3191102 ]], dtype=float32)>)

In [18]:
# Shuffle (unset global seed)
r1_shuffle = tf.random.shuffle(r1) # normal shuffle with no seed (default global is auto set), should be different between runs
# Set operational seed
r1_local = tf.random.shuffle(r1, seed=1) # shuffle with only operational seed, (default global is auto set, then additionally uses operational seed)
r1_shuffle, r1_local

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.23193765, -1.8107855 ],
        [-0.7565803 , -0.06854702],
        [ 0.09988727, -0.50998646],
        [ 0.07595026, -1.2573844 ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[ 0.07595026, -1.2573844 ],
        [ 0.09988727, -0.50998646],
        [-0.23193765, -1.8107855 ],
        [-0.7565803 , -0.06854702]], dtype=float32)>)

In [19]:
import random
tf.random.set_seed(random.randint(1, 1000)) # setting seed randomly to emulate an empty seed since the 2nd time it's set (below) keeps the set between runs in this notebook
r2_shuffle = tf.random.shuffle(r2) # Unseeded, randomized every time it's run
# Set global seed
tf.random.set_seed(42)
r2_global = tf.random.shuffle(r2) # No operational seed results in the same shuffle every run using only globally set seed
# Set operational seed
r2_local_global = tf.random.shuffle(r2, seed=1) # Uses both operational and global seed to shuffle, results in the same shuffle every run
r2_local_global_2 = tf.random.shuffle(r2, seed=2) # Using a different operational seed creates a different shuffle than 2_local_global but still remains the same between runs
r2_shuffle, r2_global, r2_local_global, r2_local_global_2

(<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[ 1.4680984 , -0.69443417],
        [ 1.0842259 , -2.3191102 ],
        [ 1.9547467 ,  0.71566796],
        [-0.56105393, -1.1563786 ],
        [ 0.61561096,  0.67493457]], dtype=float32)>,
 <tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[ 0.61561096,  0.67493457],
        [ 1.4680984 , -0.69443417],
        [ 1.0842259 , -2.3191102 ],
        [ 1.9547467 ,  0.71566796],
        [-0.56105393, -1.1563786 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[ 1.0842259 , -2.3191102 ],
        [ 1.4680984 , -0.69443417],
        [-0.56105393, -1.1563786 ],
        [ 1.9547467 ,  0.71566796],
        [ 0.61561096,  0.67493457]], dtype=float32)>,
 <tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[ 1.9547467 ,  0.71566796],
        [ 1.4680984 , -0.69443417],
        [ 0.61561096,  0.67493457],
        [ 1.0842259 , -2.3191102 ],
        [-0.56105393, -1.1563786 ]], dtype=float32)>)

In [20]:
# Create a tensor of all 1s
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [21]:
# Create a tensor of all 0s
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU

X = tf.constant(some_matrix) # Capitals are often associated with matricies or tensors

y = tf.constant(vector) # While lowercase are associated with vectors

In [22]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # Creates a NumPy array between 1 and 25
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [23]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [24]:
3 * 8

24

In [25]:
A.ndim

3

### Getting information from tensors

* **Shape**: Length (number of elements) of each of the dimensions of a tensor.
  * `tensor.shape`
* **Rank**: Number of dimensions of a tensor. 
  * Scaler = rank 0
  * Vector = rank 1
  * Matrix = rank 2 
  * Tensor = rank n
  * `tensor.ndim`
* **Axis or Dimension**: A particular dimension of a tensor.
  * `tensor[0], tensor[:, 1]...`
* **Size**: The total number of items in the tensor.
  * `tf.size(tensor)`

In [26]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [27]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor), rank_4_tensor[0]

(TensorShape([2, 3, 4, 5]),
 4,
 <tf.Tensor: shape=(), dtype=int32, numpy=120>,
 <tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
 array([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], dtype=float32)>)

In [28]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: 120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [29]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [30]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [31]:
some_list[:1] # gets the first element (0-1)

[1]

In [32]:
# Get the first dimension from each index except for the final one
rank_4_tensor[:1,:1, :1] # or rank_4_tensor[:1,:1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [33]:
rank_4_tensor[:1,:1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [34]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[1,2], [3,4]])  # alternatively => tf.zeros(shape=[2, 2])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [35]:
# Get the last item of each row of the rank_2_tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 4], dtype=int32)>

In [36]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [37]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

### Manipulating tensors (tensor operations)

***Basic operations***

`+`, `-`, `*`, `/`


In [38]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[1, 2], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[11, 12],
       [13, 14]], dtype=int32)>

In [39]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [40]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10, 20],
       [30, 40]], dtype=int32)>

In [41]:
# We can use the tensorflow built in functions
tf.multiply(tensor, 10) # using tf functions is usually faster

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10, 20],
       [30, 40]], dtype=int32)>

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are 2 rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the inner dimensions

In [42]:
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 7, 10],
       [15, 22]], dtype=int32)>

In [43]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 7, 10],
       [15, 22]], dtype=int32)>

In [44]:
X = tf.constant([[1, 2, 3], [4, 5, 6]])
Y = tf.constant([[1, 2], [3, 4], [5, 6]])
tf.matmul(X, Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[22, 28],
       [49, 64]], dtype=int32)>

In [45]:
tf.matmul(Y, X)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 9, 12, 15],
       [19, 26, 33],
       [29, 40, 51]], dtype=int32)>

In [46]:
r1 = tf.reshape(Y, shape=[3, 2])
r2 = tf.reshape(Y, shape=[2, 3])
r1, r2

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [47]:
tf.matmul(X, tf.reshape(Y, shape=(3, 2)))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[22, 28],
       [49, 64]], dtype=int32)>

In [48]:
# Can do the same with transpose except transpose also flips the axises
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 4],
        [2, 5],
        [3, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [49]:
# Try matrix multiplication
A = tf.constant([[1, 2], [3, 4], [5, 6]])
B = tf.constant([[1, 2], [3, 4], [5, 6]])
tf.matmul(A, tf.reshape(B, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 9, 12, 15],
       [19, 26, 33],
       [29, 40, 51]], dtype=int32)>

In [50]:
# With transpose rather than reshape
tf.matmul(A, tf.transpose(B))
# results in different dot product due to axis flip

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 5, 11, 17],
       [11, 25, 39],
       [17, 39, 61]], dtype=int32)>

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()` 


In [51]:
# Transposed
transposed = tf.transpose(B)
print(tf.matmul(A, transposed))
print(tf.tensordot(A, transposed, axes=1))

tf.Tensor(
[[ 5 11 17]
 [11 25 39]
 [17 39 61]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[ 5 11 17]
 [11 25 39]
 [17 39 61]], shape=(3, 3), dtype=int32)


In [52]:
# Reshaped
reshaped = tf.reshape(B, shape=(2, 3))
print(tf.matmul(A, reshaped))
print(tf.tensordot(A, reshaped, axes=1))

tf.Tensor(
[[ 9 12 15]
 [19 26 33]
 [29 40 51]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[ 9 12 15]
 [19 26 33]
 [29 40 51]], shape=(3, 3), dtype=int32)


Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to satisfy the matrix multiplication rules.

### Changing the datatype of a tensor

In [53]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
C = tf.constant([7, 10])
print(B.dtype, C.dtype)

<dtype: 'float32'> <dtype: 'int32'>


In [54]:
# Change from float32 to float16
D = tf.cast(B, dtype=tf.float16)
D.dtype


tf.float16

In [55]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E.dtype

tf.float32

In [56]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16.dtype

tf.float16

### Aggregating Tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [57]:
# Get the absolute values with `tf.abs()`
F = tf.constant(np.random.randint(-50, 50, 50))
Absolutes = tf.abs(F).numpy()
# Min
Min = tf.reduce_min(F).numpy()
# Max
Max = tf.reduce_max(F).numpy()
# Mean
Mean = tf.reduce_mean(F).numpy()
# Sum
Sum = tf.reduce_sum(F).numpy()
print("Original Tensor:\n%s\n\nAbsolute Values:\n%s\n\nMin: %s\n\nMax: %s\n\nMean: %s\n\nSum: %s"%(F.numpy(), Absolutes, Min, Max, Mean, Sum))

Original Tensor:
[ -2  39  -2  27 -16  20 -47  -9 -31  40  29  43  30  15  32 -21 -35  16
  -7  46  32  22 -15  24  24 -26  43 -11  -1  19 -16  21 -29  44  15 -11
 -12  45   3  15  -8 -43 -46   8  16  -3 -15 -11 -23 -13]

Absolute Values:
[ 2 39  2 27 16 20 47  9 31 40 29 43 30 15 32 21 35 16  7 46 32 22 15 24
 24 26 43 11  1 19 16 21 29 44 15 11 12 45  3 15  8 43 46  8 16  3 15 11
 23 13]

Min: -47

Max: 46

Mean: 4

Sum: 215


🔧 **Exercise:** With what we've just learned, find the variance and standard deviation of our `F` tensor using TensorFlow methods.

In [58]:
import tensorflow_probability as tfp
#Variance
cast = tf.cast(F, dtype=tf.float32)
Variance = tf.math.reduce_variance(cast).numpy()

#STD
Std = tf.math.reduce_std(cast).numpy()
print("Variance: %s\n\nStandard Deviation: %s\n"%(Variance, Std))

# Alternatively, use tfp.stats.variance for rounded int
VarianceTfp = tfp.stats.variance(F).numpy()
print("TFP Variance: %s"%VarianceTfp)

Variance: 665.69

Standard Deviation: 25.800968

TFP Variance: 665


### Find the positional minimum and maximum

In [59]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [60]:
# Find the positional maximum (argmax)
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [61]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [62]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [63]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [64]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [65]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [66]:
# Find the minimum using reduce_min
tf.reduce_min(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions)

In [67]:
# Create a tensor
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [68]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [69]:
G_squeezed = tf.squeeze(G)
G_squeezed.shape

TensorShape([50])

In [70]:
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G_squeezed = tf.squeeze(G)
print(G.shape, G_squeezed.shape)

(1, 1, 1, 1, 50) (50,)


### One-hot encoding tensors

In [71]:
# Create a list of indicies
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indicies
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [72]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value='true', off_value='false')

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'true', b'false', b'false', b'false'],
       [b'false', b'true', b'false', b'false'],
       [b'false', b'false', b'true', b'false'],
       [b'false', b'false', b'false', b'true']], dtype=object)>

### Squaring, Log, Square Root

In [73]:
# Create a new tensor
H = tf.range(1, 10)

# Square it
squared = tf.square(H)

# SquareRoot (method requires non-int type)
float32Range = tf.cast(H, dtype=tf.float32)
sqrt = tf.sqrt(float32Range)

# Log (method requires non-int type)
log = tf.math.log(float32Range)
print("squared:", squared.numpy(), '\n')
print("sqrt:", sqrt.numpy(), '\n')
print("log:", log.numpy(), '\n')

squared: [ 1  4  9 16 25 36 49 64 81] 

sqrt: [1.        1.4142135 1.7320508 2.        2.2360678 2.4494896 2.6457512
 2.828427  3.       ] 

log: [0.        0.6931472 1.0986123 1.3862944 1.609438  1.7917595 1.9459102
 2.0794415 2.1972246] 



### Tensors and Numpy

In [74]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [75]:
# Convert the tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [76]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [77]:
J = tf.constant([3.])
J.numpy()[0]

3.0

In [78]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

In [79]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]